# ***NBA Engine***
Via LLM Reasoning (Local LLama Model)

| Requirement                 | You Should Do                                     |
| --------------------------- | ------------------------------------------------- |
| Choose a channel            | Based on behavior, flow, and history              |
| Pick best send time         | Based on customer activity (heuristic or learned) |
| Generate a response message | Personalized or templated                         |
| Explain your reasoning      | Why this action fits this customer                |
| Output JSON                 | Fully reproducible, structured format             |
| Justify method              | Why LLM is best                 |


## **Llama 3.2 using Langchain wrapping**

In [9]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_community.chat_models import ChatOllama
from datetime import datetime, timedelta
from tqdm import tqdm
import json
import random

## **Clean Unified feed after user behaviour analysis**

In [10]:
with open(r"..\outputs\json\parsed_conversations_tagged_summary_clustered.json", "r", encoding="utf-8") as f:
    all_conversations = json.load(f)

In [11]:
llm = ChatOllama(model="llama3.2", base_url="http://localhost:11434", temperature=0.4)

## **Response Schema and Output Parser**

In [12]:
response_schemas = [
    ResponseSchema(name="customer_id", description="Customer ID"),
    ResponseSchema(name="channel", description="Best communication channel: twitter_dm_reply, scheduling_phone_call, or email_reply"),
    ResponseSchema(name="send_time", description="Best time to send message, in ISO format (UTC)"),
    ResponseSchema(name="message", description="The actual message to send"),
    ResponseSchema(name="reasoning", description="Why this channel/time/message was chosen"),
    ResponseSchema(name="resolved", description="true if customer issue seems resolved, else false")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [13]:
nba_prompt_template = PromptTemplate(
    template="""
You are an intelligent reasoning agent for Next Best Action (NBA) in customer support.

Your task is to recommend:
1. Best channel (twitter_dm_reply, scheduling_phone_call, or email_reply)
2. Best send time in UTC (consider customer activity & urgency)
3. A helpful, personalized response message
4. Reasoning behind your decision
5. Whether the issue appears resolved

---

You are provided a customer conversation flow with detailed behavior and analysis:

### Features Available

**trajectory_category** → whether user's mood is improving, worsening, or mixed/stable
**Conversation Summary:**
- Customer ID: {customer_id}
- Agent ID: {agent_id}
- Total User Turns: {num_user_turns}
- Total Agent Turns: {num_agent_turns}
- Average Sentiment Confidence: {avg_sentiment_confidence:.2f}
- Median Sentiment Confidence: {median_sentiment_confidence:.2f}
- Avg Response Gap (seconds): {avg_response_gap_seconds:.2f}
- Fine-Grained Issue: {fine_grained_issue}
- High-Level Issue: {high_level_issue}
- Sentiment Trajectory Category: {trajectory_category}
- Flow Cluster ID: {flow_cluster}

**Conversation Text Snippets**:
{conversation_text}

---

### Instructions

- Choose the most appropriate **channel**:
  - Use `twitter_dm_reply` if conversation is live and active
  - Use `scheduling_phone_call` for high-stakes/confusing/urgent problems
  - Use `email_reply` if sentiment has stabilized, or issue is resolved

- Pick best **send_time** in UTC (based on timestamps or urgency)

- Compose a helpful **message** (solve or clarify)

- Provide clear **reasoning**

- Set `resolved = true` only if user’s last message shows satisfaction

---

{format_instructions}
""",
    input_variables=[
        "customer_id", "agent_id", "num_user_turns", "num_agent_turns",
        "avg_sentiment_confidence", "median_sentiment_confidence",
        "avg_response_gap_seconds", "fine_grained_issue", "high_level_issue",
        "trajectory_category", "flow_cluster", "conversation_text"
    ],
    partial_variables={"format_instructions": format_instructions}
)


## **NBA**

In [ ]:
import copy
from dateutil import parser
from datetime import datetime, timedelta, timezone
from tqdm import tqdm

nba_records = []
nba_outputs = []

def generate_nba_instruction(convo):
    summary = convo.get("flow_summary", {})
    original_messages = copy.deepcopy(convo.get("conversation", [])) 
    messages = convo.get("conversation", [])
    customer_id = convo.get("customer_id")
    agent_id = convo.get("agent_id")
    flow_cluster = convo.get("flow_cluster", -1)


    conversation_text = "\n".join([
        f"{m['sender'].capitalize()}: {m['text']}" for m in original_messages if 'text' in m
    ])

    prompt_input = {
        "customer_id": customer_id,
        "agent_id": agent_id,
        "num_user_turns": summary.get("num_user_turns", 0),
        "num_agent_turns": summary.get("num_agent_turns", 0),
        "avg_sentiment_confidence": summary.get("avg_sentiment_confidence", 0.0),
        "median_sentiment_confidence": summary.get("median_sentiment_confidence", 0.0),
        "avg_response_gap_seconds": summary.get("avg_response_gap_seconds", 0.0),
        "fine_grained_issue": summary.get("fine_grained_issue", ""),
        "high_level_issue": summary.get("high_level_issue", ""),
        "trajectory_category": summary.get("trajectory_category", "stable"),
        "flow_cluster": flow_cluster,
        "conversation_text": conversation_text
    }

    # Prompt LLM
    prompt = nba_prompt_template.format(**prompt_input)
    response = llm.invoke(prompt)
    response_content = getattr(response, "content", None)
    if not isinstance(response_content, str):
        raise ValueError(f"Invalid LLM response type: {type(response_content)}")

    parsed_output = output_parser.parse(response_content)

    # Clamping send_time
    now_utc = datetime.now(timezone.utc)
    max_send_dt = now_utc + timedelta(days=2)

    try:
        parsed_send_dt = parser.parse(parsed_output["send_time"])
        if parsed_send_dt.tzinfo is None:
            parsed_send_dt = parsed_send_dt.replace(tzinfo=timezone.utc)
    except Exception:
        parsed_send_dt = now_utc + timedelta(hours=1)

    safe_day = min((now_utc + timedelta(days=2)).day, 28)
    final_send_dt = parsed_send_dt.replace(
        year=now_utc.year,
        month=now_utc.month,
        day=safe_day
    )
    if final_send_dt > max_send_dt:
        final_send_dt = max_send_dt

    parsed_output["send_time"] = final_send_dt.isoformat().replace("+00:00", "Z")

    # Appending new message to conversation
    convo["conversation"].append({
        "sender": "agent",
        "text": parsed_output["message"],
        "timestamp": parsed_output["send_time"]
    })

    # only original messages for chat log
    chat_log = "\n".join([
        f"{m['sender'].capitalize()}: {m['text']}" for m in original_messages
    ])

    nba_records.append({
        "customer_id": customer_id,
        "chat_log": chat_log,
        "channel": parsed_output["channel"],
        "message": parsed_output["message"],
        "send_time": parsed_output["send_time"],
        "reasoning": parsed_output["reasoning"],
        "issue_status": "resolved" if parsed_output["resolved"] else "pending_customer_response"
    })

    return convo

In [ ]:

for convo in tqdm(all_conversations[5:25]):
    try:
        nba_output = generate_nba_instruction(convo)
        nba_outputs.append(nba_output)
    except Exception as e:
        print(f"Error for customer {convo.get('customer_id', 'unknown')}: {e}")

 90%|█████████ | 18/20 [36:49<06:07, 183.55s/it]

Error for customer 245869: Got invalid return object. Expected key `resolved` to be present, but got {'customer_id': '245869', 'channel': 'email_reply', 'send_time': '2023-03-08T14:30:00+00:00', 'message': "Thank you for sharing your concerns about the iPhone notifications. We apologize for any inconvenience this has caused and are happy to help you find a solution. Can you please tell us more about what's happening with your iPhone?", 'reasoning': "Given that the customer's sentiment trajectory category is 'stable' and their fine-grained issue is 'feedback', it suggests that they have cooled down and are looking for a resolution. Since there is no urgent or high-stakes issue, an email reply is the most suitable channel to provide a helpful response without overwhelming them with a phone call."}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


100%|██████████| 20/20 [38:32<00:00, 115.64s/it]

Error for customer 122826: Got invalid return object. Expected key `resolved` to be present, but got {'customer_id': '122826', 'channel': 'twitter_dm_reply', 'send_time': '2023-03-08T14:30:00Z', 'message': "Thank you so much for your kind words about our staff and airline! We're thrilled to have made a positive impression on your first flight with us. Your loyalty means the world to us!", 'reasoning': "Given that the customer's sentiment has stabilized at 'stable' and they've expressed their satisfaction with our staff, I chose Twitter DM Reply as the best channel. The send time is 14:30 UTC, which allows for a timely response while also considering the user's recent activity. The message aims to acknowledge and extend gratitude for their positive experience, further solidifying the positive sentiment."}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


## **Saving Ouputs**

In [17]:
import pandas as pd

# Save updated conversations
with open(r"..\outputs\json\nba_conversations_updated.json", "w", encoding="utf-8") as f:
    json.dump(nba_outputs, f, indent=2)

# Save NBA table
pd.DataFrame(nba_records).to_csv(r"..\outputs\csv\nba_actions_log.csv", index=False)